In [1]:
# -----------------------------------------------------------------------------------------
# CAGED (Cadastro Geral de Empregados e Desempregados)
# type:         function                              
# filename:     caged_xlsx_to_csv_v021                
# version:      v022                                  
# date:         29/03/2022 - 02/04/2022               
# description:  process caged xlsx file and save to csv file
# -----------------------------------------------------------------------------------------


# imports and variable's names declarations
# -----------------------------------------------------------------------------------------
import pandas as pd

# lists
# months names
months_names_br = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                   'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
month_names_dec = ('01',      '02',        '03',    '04',    '05',
                   '06',    '07',    '08',     '09',       '10',      '11',       '12')

# regions list
regions_table = ('Região e UF', 'Unnamed: 1_level_1')
regions_names = ('Norte', 'Nordeste', 'Sudeste',
                 'Sul', 'Centro-Oeste', 'Não identificado')

# sectors list
gr_act_econ_table = (
    'Grupamento de Atividades Econômicas e Seção CNAE 2.0', 'Unnamed: 1_level_1')
sector_names = ('Agricultura, pecuária, produção florestal, pesca e aquicultura', 'Indústria geral', 'Construção',
                'Comércio; reparação de veículos automotores e motocicletas', 'Serviços', 'Não identificado***')

# columns names list
first_columns_names_out = {'Mês': '', 'Saldos': 'total'}
second_columns_names_out = {1: 'reg1', 9: 'reg2',
                            19: 'reg3', 24: 'reg4', 28: 'reg5', 33: 'reg9'}
third_columns_names_out = {1: 'setA', 2: 'setE',
                           7: 'setF', 8: 'setG', 9: 'setU', 27: 'setZ'}

# variables
file_in = '3-tabelas_JAN.xlsx'
file_out = 'caged.csv'



# functions declarations
# -----------------------------------------------------------------------------------------
# function:     try_int: (data value) -> boolean
# description:  search for invalid types
def try_int(val):
    try:
        val = float(val)
        val = int(val)
        return True
    except:
        return False

# function:     getColumnsList: (dataframe) -> list
# description:  creates a dataframe columns list
def getColumnsList(param_df):
    columns_list = []
    column_names = param_df.columns.values.tolist()
    for column_name in column_names:
        # column_name has 2 parts (date, info_type)
        col_date, col_info = column_name
        month = col_date.split('/')[0]
        if month in months_names_br:
            if col_info.lower().find('saldos') != -1:
                columns_list.append(column_name)

    return(columns_list)

# function:     cleanList: (dataframe, list) -> list
# description:  remove invalid values
def cleanList(cl_df2, list_to_clean):
    for month in list_to_clean:
        filter = cl_df2[month].apply(try_int)
        df_tmp = cl_df2[month]
        df_tmp.transpose
        cl_df2.drop(df_tmp[filter == False].index, inplace=True)

    return(cl_df2)

# function:     processSheetData: (filename, sheetname, list, list)
# description:  create data values by regions
def processSheetData(sheetfile, sheetname, columnnamelist, columnsnameslist ):
    tmpdf = pd.read_excel(sheetfile, sheet_name=sheetname, skiprows=4, header=[0, 1])

    # get months's columns list
    monthslist = getColumnsList(tmpdf)
    tmpdf = cleanList(tmpdf, monthslist)

    # select specifics columns
    tmpauxdf = tmpdf[tmpdf[columnnamelist].isin(columnsnameslist)]
    out_df = tmpauxdf[monthslist]

    out_df = tmpauxdf[monthslist].astype(int)
    out_df = out_df.transpose()

    return(out_df)



# -----------------------------------------------------------------------------------------
# init
# -----------------------------------------------------------------------------------------

# first datasheet: process data values by date
first_df = pd.read_excel(file_in, sheet_name=7, skiprows=4)
df2 = first_df.copy()

# select date cells only
df2 = df2[(df2['Mês'].str.slice(-4) >= '1900') &
          (df2['Mês'].str.slice(-4) <= '2099')]

# replacements months's portuguese names to months's numbers names
mbr2mdec = dict(zip(months_names_br, month_names_dec))
df2['Mês'] = df2['Mês'].replace(mbr2mdec, regex=True)
df2['Mês'] = df2['Mês'].str.replace('/', '-')
df2['Mês'] = pd.to_datetime(
    df2['Mês'], format='%Y-%m-%d', infer_datetime_format=True)

# filter invalid types
filter = df2['Saldos'].apply(try_int)
df2.drop(df2[filter == False].index, inplace=True)

df2['Saldos'] = df2['Saldos'].astype(int)
first_out_df = df2[['Mês', 'Saldos']]

# second datasheet: process data values by regions
second_out_df = processSheetData(file_in, 11, regions_table, regions_names)

# third datasheet: process data values by sectors
third_out_df = processSheetData(file_in, 9, gr_act_econ_table, sector_names)

# final datasheet: concatenate all datasheets and creates csv file
temp_first_out_df = first_out_df.reset_index(drop=True)
temp_first_out_df.rename(columns=first_columns_names_out, inplace=True)

temp_second_out_df = second_out_df.reset_index(drop=True)
temp_second_out_df.rename(columns=second_columns_names_out, inplace=True)

temp_third_out_df = third_out_df.reset_index(drop=True)
temp_third_out_df.rename(columns=third_columns_names_out, inplace=True)

final_out_df = pd.concat(
    [temp_first_out_df, temp_second_out_df, temp_third_out_df], axis=1)
final_out_df.to_csv(file_out, index=False)

